In [ ]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix==0.0.4

  Preparing metadata (setup.py) ... done


In [ ]:
!python -m pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
import wandb
from wandb.keras import WandbCallback
from dlomix.data import RetentionTimeDataset
import tensorflow as tf
from dlomix.eval import TimeDeltaMetric

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
BATCH_SIZE = 64

rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, batch_size=BATCH_SIZE, val_ratio=0.2, test=False, sequence_col= "sequence", target_col= "irt")

In [ ]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['whole_dataset_3.csv']))
df3 = df2.head(10000)

KeyError: ignored

In [ ]:

seq = df3["modified_sequence"].to_numpy(dtype = "S")
targ = df3["indexed_retention_time"].to_numpy(dtype = "f")

NameError: ignored

In [ ]:
# enter project name
project_name = 'prosit_eval'
wandb.init(project=project_name)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
df = pd.read_csv("dataset_median/whole_dataset.csv")

In [ ]:
df.head()

,Unnamed: 0,modified_sequence,indexed_retention_time
0,0,AAAASAAEAGIATTGTEGER,31.935811
1,1,AAAC[UNIMOD:4]FFEQPPR,53.785709
2,2,AAADFATHGK,4.344702
3,3,AAADLMAYC[UNIMOD:4]EAHAK,63.955956
4,4,AAADSDPNLDPLMNPHIR,63.854205


In [ ]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [ ]:
df.head()

,modified_sequence,indexed_retention_time
0,AAAASAAEAGIATTGTEGER,31.935811
1,AAAC[UNIMOD:4]FFEQPPR,53.785709
2,AAADFATHGK,4.344702
3,AAADLMAYC[UNIMOD:4]EAHAK,63.955956
4,AAADSDPNLDPLMNPHIR,63.854205


In [ ]:
df.to_csv("whole_dataset_3.csv",index=False)

In [ ]:
df

,Unnamed: 0,modified_sequence,indexed_retention_time
0,0,AAAASAAEAGIATTGTEGER,31.935811
1,1,AAAC[UNIMOD:4]FFEQPPR,53.785709
2,2,AAADFATHGK,4.344702
3,3,AAADLMAYC[UNIMOD:4]EAHAK,63.955956
4,4,AAADSDPNLDPLMNPHIR,63.854205
...,...,...,...
881710,881710,YVAIC[UNIMOD:4]NPLRYPVIM[UNIMOD:35]NR,68.012109
881711,881711,YVAIC[UNIMOD:4]SPLRYPVIMSK,74.275623
881712,881712,YVAIC[UNIMOD:4]SPLRYPVIM[UNIMOD:35]SK,72.702211
881713,881713,YVLTSPRSLEAC[UNIMOD:4]AR,46.175652


In [ ]:

# csv import fails for some reason -> try tuple of np arrays -> need to specify column names or use default names "sequence" and "irt" in csv
BATCH_SIZE = 64

rtdata = RetentionTimeDataset(data_source=(seq, targ), sequence_col= "modified_sequence", target_col= "indexed_retention_time",
                              seq_length=30, batch_size=BATCH_SIZE, val_ratio=0.2, test=False)


In [1]:
rtdata.batch_size

NameError: ignored

In [ ]:
len(rtdata.train_data) * rtdata.batch_size

1088

In [ ]:
len(rtdata.val_data) * rtdata.batch_size

320

In [ ]:
from dlomix.models import PrositRetentionTimePredictor
model = PrositRetentionTimePredictor(seq_length=30)


In [3]:
# Enter weights and biases run name.
project_name = 'prosit_eval'
wandb.init(project=project_name, name='colab_test_8')

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mean_absolute_error', TimeDeltaMetric()])

NameError: ignored

In [4]:
config = wandb.config
config.seq_length = 30
config.batch_size = BATCH_SIZE
config.val_ratio = 0.2

NameError: ignored

In [ ]:
config.lr = 0.0001
config.optimizer = "adam"

In [ ]:
# Try programmitically creating a report
import wandb.apis.reports as wr
report = wr.Report(project=project_name,
                   title = "Second WANDB Report",
                   description = "What an easy way too create a report!")

In [ ]:
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=3, callbacks=[WandbCallback()] )
from dlomix.reports import RetentionTimeReport

In [ ]:
# here we pass the WandbCallback to model.fit

history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=3, callbacks=[WandbCallback()] )

report.blocks = [
    wr.TableOfContents(),
    wr.H1("Text and images example"),
    wr.P("Lorem ipsum dolor sit amet. Aut laborum perspiciatis sit odit omnis aut aliquam voluptatibus ut rerum molestiae sed assumenda nulla ut minus illo sit sunt explicabo? Sed quia architecto est voluptatem magni sit molestiae dolores. Non animi repellendus ea enim internos et iste itaque quo labore mollitia aut omnis totam."),
    wr.Image('https://api.wandb.ai/files/telidavies/images/projects/831572/8ad61fd1.png', caption='Craiyon generated images'),
    wr.P("Et voluptatem galisum quo facilis sequi quo suscipit sunt sed iste iure! Est voluptas adipisci et doloribus commodi ab tempore numquam qui tempora adipisci. Eum sapiente cupiditate ut natus aliquid sit dolor consequatur?"),
]
report.save()
wandb.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/3
15/17 [=========================>....] - ETA: 0s - loss: 3067.4834 - mean_absolute_error: 45.2828 - timedelta: 51.0952

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230706_111307-9tnruzzi/files/model-best)... Done. 0.1s


17/17 [==============================] - 34s 1s/step - loss: 3546.3867 - mean_absolute_error: 48.9679 - timedelta: 58.5058 - val_loss: 1317.4846 - val_mean_absolute_error: 29.0632 - val_timedelta: 20.3524
Epoch 2/3
16/17 [===========================>..] - ETA: 0s - loss: 3271.4946 - mean_absolute_error: 46.8798 - timedelta: 54.0790

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230706_111307-9tnruzzi/files/model-best)... Done. 0.1s


17/17 [==============================] - 18s 1s/step - loss: 3393.2856 - mean_absolute_error: 47.8337 - timedelta: 57.1826 - val_loss: 992.5397 - val_mean_absolute_error: 24.5619 - val_timedelta: 18.9521
Epoch 3/3
16/17 [===========================>..] - ETA: 0s - loss: 1387.8639 - mean_absolute_error: 29.4860 - timedelta: 33.9429

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230706_111307-9tnruzzi/files/model-best)... Done. 0.1s


17/17 [==============================] - 18s 1s/step - loss: 1423.4786 - mean_absolute_error: 29.9308 - timedelta: 35.7604 - val_loss: 913.3253 - val_mean_absolute_error: 25.2885 - val_timedelta: 31.9956


NameError: ignored